In [22]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import folium 
from datetime import datetime
from math import sin, cos, sqrt, atan2, radians


In [23]:
data = pd.read_csv('ChargingStationCleaned.csv')
data['year'] = pd.to_datetime(data['commissioning_date']).dt.year

In [24]:
data.head()

,operator,address,house_number,placeholder1,postcode,city,federal_state,metropolitan_area,latitude_[dg],longitude_[dg],...,number_of_charging_points,type_of_plug_1,p1_[kw],type_of_plug_2,p2_[kw],type_of_plug_3,p3_[kw],type_of_plug_4,p4_[kw],year
0,EnBW mobility+ AG und Co.KG,Hauptstraï¿½e,91c,NaN,73340,Amstetten,Baden-Wï¿½rttemberg,Landkreis Alb-Donau-Kreis,48.578534,9.874840,...,2,AC Kupplung Typ 2,43.0,"DC Kupplung Combo, DC CHAdeMO",50.0,0,0.0,0,0.0,2019
1,SWU Energie GmbH,Lindenstraï¿½e,7,NaN,89176,Asselfingen,Baden-Wï¿½rttemberg,Landkreis Alb-Donau-Kreis,48.528983,10.193397,...,1,AC Steckdose Typ 2,22.0,0,0.0,0,0.0,0,0.0,2019
2,SWU Energie GmbH,ï¿½llengasse,10,NaN,89177,Ballendorf,Baden-Wï¿½rttemberg,Landkreis Alb-Donau-Kreis,48.553539,10.082678,...,1,AC Steckdose Typ 2,22.0,AC Steckdose Typ 2,22.0,0,0.0,0,0.0,2019
3,SWU Energie GmbH,Sterngasse,6,NaN,88481,Balzheim,Baden-Wï¿½rttemberg,Landkreis Alb-Donau-Kreis,48.179959,10.076977,...,1,AC Steckdose Typ 2,22.0,0,0.0,0,0.0,0,0.0,2022
4,EnBW mobility+ AG und Co.KG,Heuweg,12,NaN,89179,Beimerstetten,Baden-Wï¿½rttemberg,Landkreis Alb-Donau-Kreis,48.480387,9.980588,...,2,"AC Steckdose Typ 2, AC Schuko",22.0,"AC Steckdose Typ 2, AC Schuko",22.0,0,0.0,0,0.0,2019


In [26]:
import sqlite3
import requests
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import geopandas as gpd

def get_osrm_distance(lat1, lon1, lat2, lon2):
    # Verifica se la distanza è già stata calcolata e salvata nel database
    conn = sqlite3.connect('distances.db', isolation_level=None, check_same_thread=False)
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS distances
                      (lat1 REAL, lon1 REAL, lat2 REAL, lon2 REAL, distance REAL)''')
    conn.commit()
    
    cursor.execute("SELECT distance FROM distances WHERE lat1=? AND lon1=? AND lat2=? AND lon2=?",
                   (lat1, lon1, lat2, lon2))
    result = cursor.fetchone()
    
    if result is not None:
        distance = result[0]
    else:
        url = f"http://router.project-osrm.org/route/v1/driving/{lon1},{lat1};{lon2},{lat2}?overview=false"
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            if 'routes' in data and len(data['routes']) > 0:
                distance = data['routes'][0]['distance']
                # Salva la distanza nel database
                cursor.execute("INSERT INTO distances (lat1, lon1, lat2, lon2, distance) VALUES (?, ?, ?, ?, ?)",
                               (lat1, lon1, lat2, lon2, distance))
                conn.commit()
        else:
            distance = None

    conn.close()
    return distance

In [27]:
import requests
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import geopandas as gpd

def create_network(year, max_distance=100000):  # max_distance in metri
    data_year = data[data['year'] <= year]
    n = len(data_year)
    positions = data_year[['latitude_[dg]', 'longitude_[dg]']].to_numpy()

    network = nx.Graph()
    network.add_nodes_from(range(1, n + 1))

    for i in range(n):
        for j in range(i + 1, n):  # Evita di ripetere gli archi
            distance = get_osrm_distance(positions[i, 0], positions[i, 1], positions[j, 0], positions[j, 1])
            if distance is not None and distance < max_distance:
                network.add_edge(i + 1, j + 1, weight=distance)

    pos = {i + 1: positions[i] for i in range(n)}

    options = {
        'node_color': 'lavender',
        'node_size': 30,
        'width': 1,
        'arrowstyle': '->',
        'arrowsize': 1,
    }

    plt.clf()
    plt.cla()
    nx.draw_networkx(network, pos=pos, **options)
    plt.title(f'EV charging stations network ({year}) of BEV with 100 km range', fontsize=20)
    plt.show()

    nx.write_graphml(network, f"network_{year}_{max_distance}.graphml")

In [28]:
create_network(2012)

KeyboardInterrupt: 

In [ ]:
create_network(2017)

In [ ]:
create_network(2022)